In [1]:
import numpy as np

d=1

r = np.random.randint(1, 10)
r = 2

# Define x_mu
x_mu = np.array([(2 * mu - 1) / (2 * r) * np.pi for mu in range(1, r + 1)])
# x_mu = x_mu + np.pi / (2 * r) * (1 / n)

Omegas = np.arange(1, r+1)
p0 = Omegas ** d * (-1 if d % 4 == 3 else 1)

# Create matrix A
A = np.array([[np.sin(n * x) for n in range(1, r + 1)] for x in x_mu])

# Calculate the inverse of A and the dot product with p0
b = np.dot(np.linalg.inv(A).T, p0)
b = np.hstack((0.5 * b, -0.5 * b))

l1norm = np.linalg.norm(b, ord=1)
print(l1norm)
print(np.isclose(l1norm, r, atol=1e-10))

2.0
True


In [2]:
x = x_mu
# A = np.sin(np.outer(x, Omegas))
# A_1 = np.cos(np.outer(x, Omegas)) * Omegas
# b = np.dot(np.linalg.inv(A).T, p0)
# subgrad =  np.outer(b, np.sign(b)) @ np.linalg.inv(A).T @ A_1.T
# subgrad = - np.diag(subgrad)
# subgrad

# 定义解析次梯度函数（只是其中一个次梯度）
def analytical_subgradient(x, Omegas):
    p = Omegas ** d * (-1 if d % 4 == 3 else 1)
    A = np.sin(np.outer(x, Omegas))
    A_1 = np.cos(np.outer(x, Omegas)) * Omegas
    A_inv = np.linalg.inv(A)
    b = np.linalg.inv(A.T) @ p
    subgrad = -np.diag(np.outer(b, np.sign(b)) @ A_inv.T @ A_1.T)
    print((A_1 @ A_inv).T)
    print(np.sign(b))
    print(A.T @ np.sign(b))
    return subgrad

analytical_subgradient(x, Omegas)

[[ 0.5 -0.5]
 [ 0.5 -0.5]]
[ 1. -1.]
[0. 2.]


array([-2.71746121e-16, -1.03799854e-16])

In [3]:
# 计算次梯度

import autograd.numpy as np
from autograd import jacobian

# 参数
r = 2
d = 1

# 定义 Omegas 和 p0
Omegas = np.arange(1, r + 1)
p0 = Omegas ** d * (-1 if d % 4 == 3 else 1)

# 目标函数，最小化 l1 范数
def objective(x_mu):
    # 基于当前 x_mu 创建矩阵 A
    A = np.array([[np.sin(n * x) for n in range(1, r + 1)] for x in x_mu])
    
    # 计算 A 的逆和 p0 的点积
    b = np.dot(np.linalg.inv(A).T, p0)
    b = np.hstack((0.5 * b, -0.5 * b))
    
    # 返回 l1 范数
    return np.linalg.norm(b, ord=1)

# 计算次梯度的函数
def subgradient(objective, x_mu):
    # 计算 l1 范数的 sign 值
    A = np.array([[np.sin(n * x) for n in range(1, r + 1)] for x in x_mu])
    b = np.dot(np.linalg.inv(A).T, p0)
    b = np.hstack((0.5 * b, -0.5 * b))
    sign_b = np.sign(b)
    
    # 定义内部函数用于求 Jacobian
    def g(x_mu):
        A = np.array([[np.sin(n * x) for n in range(1, r + 1)] for x in x_mu])
        b = np.dot(np.linalg.inv(A).T, p0)
        b = np.hstack((0.5 * b, -0.5 * b))
        return b
    
    # 计算 g(x_mu) 的 Jacobian 矩阵
    jacobian_g = jacobian(g)(x_mu)
    
    # 通过链式法则组合次梯度
    subgrad = np.dot(sign_b, jacobian_g)
    
    return subgrad

# 示例 x_mu
x_mu = np.random.rand(r)

# 计算次梯度
sg = subgradient(objective, x_mu)
print("次梯度为:", sg)


次梯度为: [-21.10524141  -1.0849161 ]


In [4]:
analytical_subgradient(x_mu, Omegas)

[[ 4.18392827 -8.79162908]
 [ 0.17767221  3.29443367]]
[ 1. -1.]
[-0.54090591 -0.59785263]


array([-21.10524141,  -1.0849161 ])

In [12]:
import numpy as np
from scipy.optimize import minimize

# Define r and p0
# r = np.random.randint(1, 10)

r = 5

d = 1

Omegas = np.arange(1, r+1)
p0 = Omegas ** d * (-1 if d % 4 == 3 else 1)

# Objective function to minimize the l1 norm
def objective(x_mu):
    # Create matrix A based on current x_mu
    A = np.array([[np.sin(n * x) for n in range(1, r + 1)] for x in x_mu])
    
    # Calculate the inverse of A and the dot product with p0
    # b = np.dot(np.linalg.inv(A).T, p0)
    b = np.linalg.solve(A.T, p0)
    b = np.hstack((0.5 * b, -0.5 * b))
    
    # Return the l1 norm
    # 使用 l1 范数
    # return np.sum(np.abs(b))
    # 使用 l1 范数的平方
    # return np.sum(np.abs(b))**2
    # 使用 l2 范数
    # return np.sqrt(np.sum(np.abs(b)**2))
    # 使用 l2 范数的平方
    # return np.sum(np.abs(b)**2)

# Initial guess for x_mu
x_mu = np.array([(2 * mu - 1) / (2 * r) * np.pi for mu in range(1, r + 1)])
x_mu_initial = np.random.uniform(0,  np.pi, r)

# Define bounds for optimization (0 to pi for each variable)
bounds = [(0.1, np.pi-0.1) for _ in range(r)]

from scipy.optimize import differential_evolution, basinhopping, shgo

# Perform optimization
# result = minimize(objective, x_mu_initial, method='BFGS')
# result = basinhopping(objective, x_mu_initial, niter=100)

# differential_evolution 比较稳定
result = differential_evolution(objective, bounds)
# Perform optimization with bounds
# result = minimize(objective, x_mu_initial, method='SLSQP', bounds=bounds)

# Optimal x_mu and corresponding minimized l1 norm
optimal_x_mu = result.x
optimal_x_mu.sort()
minimized_l1norm = result.fun

# Output the results
print("Optimal x_mu:", optimal_x_mu)
print("Minimized l1 norm:", minimized_l1norm)
print("x_mu:", x_mu)
print(np.isclose(minimized_l1norm, r, atol=1e-6))

Optimal x_mu: [0.40695066 1.14230402 1.70413861 2.27989935 2.85441047]
Minimized l1 norm: 6.818199803513024
x_mu: [0.31415927 0.9424778  1.57079633 2.19911486 2.82743339]
False


In [ ]:
import numpy as np
# even 函数前的系数

d=2
# Define random r
r = np.random.randint(1, 10)
# r = 5

# Define x_mu
# x_mu = np.array([0] + [(mu * np.pi) / r  for mu in range(1, r)] + [np.pi])
x_mu = np.array([(mu * np.pi) / r  for mu in range(0, r+1)])

Omegas = np.arange(1, r+1)
# q0 = np.array([0] + [-mu**2 for mu in range(1, r + 1)])
q0 = Omegas ** d * (-1 if d % 4 == 2 else 1)
q0 = np.insert(q0, 0, 1 if d == 0 else 0)

# Create matrix A
A = np.array([[1] + [np.cos(n * x) for n in range(1, r + 1)] for x in x_mu])
# A = np.cos(np.outer(x_mu, np.insert(Omegas, 0, 0)))

# Calculate the inverse of A and the dot product with p0
b = np.dot(np.linalg.inv(A).T, q0)
b = np.hstack((0.5 * b, 0.5 * b))

l1norm = np.linalg.norm(b, ord=1)
print(l1norm)
print(np.isclose(l1norm, r**2, atol=1e-10))


In [ ]:
import numpy as np
from scipy.optimize import minimize

# Define r and p0
r = np.random.randint(1, 10)
r = 5

d=4
Omegas = np.arange(1, r+1)
q0 = Omegas ** d * (-1 if d % 4 == 2 else 1)
q0 = np.insert(q0, 0, 1 if d == 0 else 0)

# Objective function to minimize the l1 norm
def objective(x_mu):
    # Create matrix A based on current x_mu
    A = np.array([[1] + [np.cos(n * x) for n in range(1, r + 1)] for x in x_mu])
    
    # Calculate the inverse of A and the dot product with p0
    b = np.dot(np.linalg.inv(A).T, q0)
    b = np.hstack((0.5 * b, 0.5 * b))
    
    # Return the l1 norm
    return np.linalg.norm(b, ord=1)

# Initial guess for x_mu
x_mu_initial = np.array([(2 * mu - 1) / (2 * r) * np.pi for mu in range(1, r + 1)])
x_mu_initial = np.random.uniform(0,  np.pi, r+1)

# Define bounds for optimization (0 to pi for each variable)
bounds = [(0, np.pi) for _ in range(r)]

# Perform optimization
result = minimize(objective, x_mu_initial, method='BFGS')

# Perform optimization with bounds
# result = minimize(objective, x_mu_initial, method='SLSQP', bounds=bounds)

# Optimal x_mu and corresponding minimized l1 norm
optimal_x_mu = result.x
minimized_l1norm = result.fun

# Output the results
print("Optimal x_mu:", optimal_x_mu)
print("Minimized l1 norm:", minimized_l1norm)

print(np.isclose(minimized_l1norm, r**2, atol=1e-10))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

r = 2
d = 1

Omegas = np.arange(1, r+1)
p0 = Omegas ** d * (-1 if d % 4 == 3 else 1)

def objective(x_mu):
    A = np.array([[np.sin(n * x) for n in range(1, r + 1)] for x in x_mu])
    
    # 添加一个小的扰动以避免奇异性
    epsilon = 1e-10
    A = A + epsilon * np.eye(r)
    
    # 使用 pseudo-inverse 而不是普通的 inverse
    A_inv = np.linalg.pinv(A)
    
    b = np.dot(A_inv.T, p0)
    b = np.hstack((0.5 * b, -0.5 * b))
    return np.linalg.norm(b, ord=1)

# Create a grid of points
n_points = 50  # 减少点的数量以提高性能
x = np.linspace(0, np.pi, n_points)
y = np.linspace(0, np.pi, n_points)
X, Y = np.meshgrid(x, y)

# Calculate Z values
Z = np.zeros((n_points, n_points))
for i in range(n_points):
    for j in range(n_points):
        try:
            Z[i, j] = objective([X[i, j], Y[i, j]])
        except np.linalg.LinAlgError:
            Z[i, j] = np.nan

# Create the 3D plot
plt.ion()  # 打开交互模式
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none')

# Set labels and title
ax.set_xlabel('x_mu[0]')
ax.set_ylabel('x_mu[1]')
ax.set_zlabel('Objective function value')
ax.set_title('Interactive 3D surface plot of the objective function')

# Add a color bar
fig.colorbar(surf, shrink=0.5, aspect=5)

# Print some statistics about the function values
print(f"Minimum value: {np.nanmin(Z)}")
print(f"Maximum value: {np.nanmax(Z)}")
print(f"Mean value: {np.nanmean(Z)}")

plt.show()

# 保持图形窗口打开，直到用户关闭它
input("Press Enter to close the figure...")

## 验证是凸函数！


In [ ]:
# r=2

import numpy as np
import matplotlib.pyplot as plt

# 定义 A(x) 矩阵
def A(x):
    return np.array([[np.sin(x[0]), np.sin(2 * x[0])],
                     [np.sin(x[1]), np.sin(2 * x[1])]])

# 定义目标函数
def objective(x):
    try:
        A_inv_T = np.linalg.inv(A(x)).T
        p = np.array([1, 2])
        b = np.dot(A_inv_T, p)
        # return np.sum(np.abs(b))
        return np.sum(b**2)
    except np.linalg.LinAlgError:
        # 如果矩阵不可逆，则返回一个大的数值
        return np.inf

# 定义凸性测试函数
def test_convexity(x1, x2, alphas):
    for alpha in alphas:
        x_alpha = alpha * x1 + (1 - alpha) * x2
        f_x_alpha = objective(x_alpha)
        f_alpha = alpha * objective(x1) + (1 - alpha) * objective(x2)
        if f_x_alpha > f_alpha + 1e-6:  # 考虑到数值误差，加入一个小的容差
            return False
    return True

# 设置随机测试的次数
num_tests = 100  # 你可以根据需要调整测试次数
convex_count = 0  # 记录满足凸性条件的次数
alphas = np.linspace(0, 1, 10)  # 用于凸性测试的 alpha 值

# 开始随机测试
for i in range(num_tests):
    # 随机生成两个点，确保在 (0, pi) 内且第二个成分大于第一个成分
    x1 = np.random.uniform(0.01, np.pi - 0.01, 2)
    x1.sort()  # 确保第二个成分大于第一个成分
    
    x2 = np.random.uniform(0.01, np.pi - 0.01, 2)
    x2.sort()  # 确保第二个成分大于第一个成分

    # 进行凸性测试
    if test_convexity(x1, x2, alphas):
        convex_count += 1

# 输出测试结果
print(f"在 {num_tests} 次随机测试中，目标函数在 {convex_count} 次测试中满足凸性条件。")


In [ ]:
# r=3

import numpy as np
import matplotlib.pyplot as plt


# 定义 A(x) 矩阵
def A(x):
    r = 3  # 设置为3个变量
    return np.array([[np.sin(i * x[j]) for i in range(1, r+1)] for j in range(r)])

# 定义目标函数
def objective(x):
    try:
        A_inv_T = np.linalg.inv(A(x)).T
        p = np.array([1, 2, 3])  # 扩展到3维
        b = np.dot(A_inv_T, p)
        # return np.sum(np.abs(b))
        return np.sum(b**2)
    except np.linalg.LinAlgError:
        # 如果矩阵不可逆，则返回一个大的数值
        return np.inf

# 定义凸性测试函数
def test_convexity(x1, x2, alphas):
    for alpha in alphas:
        x_alpha = alpha * x1 + (1 - alpha) * x2
        f_x_alpha = objective(x_alpha)
        f_alpha = alpha * objective(x1) + (1 - alpha) * objective(x2)
        if f_x_alpha > f_alpha + 1e-6:  # 考虑到数值误差，加入一个小的容差
            return False
    return True

# 设置随机测试的次数
num_tests = 1000  # 增加测试次数以获得更可靠的结果
convex_count = 0  # 记录满足凸性条件的次数
alphas = np.linspace(0, 1, 20)  # 用于凸性测试的 alpha 值

# 开始随机测试
for i in range(num_tests):
    # 随机生成两个点，确保在 (0, pi) 内且各分量按升序排列
    x1 = np.random.uniform(0.1, np.pi - 0.1, 3)
    x1.sort()  # 确保分量按升序排列
    
    x2 = np.random.uniform(0.1, np.pi - 0.1, 3)
    x2.sort()  # 确保分量按升序排列

    # 进行凸性测试
    if test_convexity(x1, x2, alphas):
        convex_count += 1

# 输出测试结果
print(f"在 {num_tests} 次随机测试中，目标函数在 {convex_count} 次测试中满足凸性条件。")
print(f"凸性比例: {convex_count/num_tests*100:.2f}%")

# 可视化（注意：3D情况下我们无法直接可视化整个函数）
# 我们可以选择固定一个变量，然后可视化其他两个变量的关系

def plot_2d_slice(fixed_var_index, fixed_value):
    x = np.linspace(0.01, np.pi - 0.01, 100)
    y = np.linspace(0.01, np.pi - 0.01, 100)
    X, Y = np.meshgrid(x, y)
    Z = np.zeros_like(X)

    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            point = [0, 0, 0]
            point[fixed_var_index] = fixed_value
            point[(fixed_var_index + 1) % 3] = X[i, j]
            point[(fixed_var_index + 2) % 3] = Y[i, j]
            Z[i, j] = objective(point)

    plt.figure(figsize=(10, 8))
    plt.contourf(X, Y, Z, levels=20, cmap='viridis')
    plt.colorbar(label='Objective Value')
    plt.title(f'2D Slice of Objective Function (x{fixed_var_index+1} = {fixed_value:.2f})')
    plt.xlabel(f'x{(fixed_var_index + 1) % 3 + 1}')
    plt.ylabel(f'x{(fixed_var_index + 2) % 3 + 1}')
    plt.show()

# 绘制三个2D切片
for i in range(3):
    plot_2d_slice(i, np.pi/2)

## 任意维度下测试凸性


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

# 定义 A(x) 矩阵
def A(x):
    r = len(x)
    return np.array([[np.sin(i * x[j]) for i in range(1, r+1)] for j in range(r)])

# 定义目标函数
def objective(x):
    try:
        A_matrix = A(x)
        p = np.arange(1, len(x) + 1)  # 生成 [1, 2, ..., n] 的向量  
        # 使用 scipy 的 solve 函数代替直接求逆
        b = linalg.solve(A_matrix.T, p, assume_a='gen')
        # return np.sum(np.abs(b))
        return np.sum(b**2)
    except np.linalg.LinAlgError:
        # 如果矩阵不可逆，则返回一个大的数值
        return np.inf

# 定义凸性测试函数
def test_convexity(x1, x2, alphas):
    for alpha in alphas:
        x_alpha = alpha * x1 + (1 - alpha) * x2
        f_x_alpha = objective(x_alpha)
        f_alpha = alpha * objective(x1) + (1 - alpha) * objective(x2)
        if f_x_alpha > f_alpha + 1e-6:  # 考虑到数值误差，加入一个小的容差
            return False, alpha, f_x_alpha, f_alpha
    return True, None, None, None

# 设置参数
r = 3  # 设置问题的维度
num_tests = 20000  # 增加测试次数以获得更可靠的结果
convex_count = 0  # 记录满足凸性条件的次数
alphas = np.linspace(0, 1, 10)  # 用于凸性测试的 alpha 值

# 开始随机测试
for i in range(num_tests):
    # 随机生成两个点，确保在 (0, pi) 内且各分量按升序排列
    x1 = np.random.uniform(0.1, np.pi - 0.1, r)
    x1.sort()  # 确保分量按升序排列
    
    x2 = np.random.uniform(0.1, np.pi - 0.1, r)
    x2.sort()  # 确保分量按升序排列

    # 进行凸性测试
    is_convex, alpha, f_x_alpha, f_alpha = test_convexity(x1, x2, alphas)
    if is_convex:
        convex_count += 1
    else:
        print(f"\n不满足凸性条件的情况 #{i+1}:")
        print(f"x1: {x1}")
        print(f"x2: {x2}")
        print(f"α: {alpha}")
        print(f"f(αx1 + (1-α)x2) = {f_x_alpha}")
        print(f"αf(x1) + (1-α)f(x2) = {f_alpha}")
        print(f"差值: {f_x_alpha - f_alpha}")

# 输出测试结果
print(f"维度: {r}")
print(f"在 {num_tests} 次随机测试中，目标函数在 {convex_count} 次测试中满足凸性条件。")
print(f"凸性比例: {convex_count/num_tests*100:.2f}%")

In [67]:
# 有一些更先进的数值微分库可以提供更高的精度。我们可以使用 jax 库，它提供了自动微分功能，通常比有限差分法更精确。


import jax
import jax.numpy as jnp
from jax import grad, hessian
import numpy as np

# 启用 64 位精度
jax.config.update("jax_enable_x64", True)

d = 1

# 定义 A(x) 矩阵
def A(x):
    r = len(x)
    return jnp.array([[jnp.sin(i * x[j]) for i in range(1, r+1)] for j in range(r)])

# 定义目标函数
def objective(x):
    A_matrix = A(x)
    Omegas = np.arange(1, len(x) + 1)
    p = jnp.array(Omegas ** d * (-1 if d % 4 == 3 else 1))
    # p = jnp.array(np.random.uniform(0, 1, len(x)))
    try:
        b = jnp.linalg.solve(A_matrix.T, p)
        # 使用 l1 范数
        # return jnp.sum(jnp.abs(b))
        # 使用 l1 范数的平方
        # return jnp.sum(jnp.abs(b))**2
        # 使用 l2 范数
        # return jnp.sqrt(jnp.sum(jnp.abs(b)**2))
        # 使用 l2 范数的平方
        return jnp.sum(jnp.abs(b)**2)
    except:
        return jnp.inf

# 使用 JAX 计算 Hessian
hessian_func = jax.jit(hessian(objective))

def is_positive_semidefinite(matrix):
    eigenvalues = jnp.linalg.eigvals(matrix)
    return jnp.all(jnp.real(eigenvalues) >= -1e-10)

# 生成随机测试点
def generate_test_points(n, dim):
    points = []
    for _ in range(n):
        while True:
            point = np.random.uniform(0, np.pi, dim)
            if len(set(point)) == dim:  # Check if all components are unique
                points.append(point)
                break
    return points
# 主测试函数
def test_hessian_semidefinite(n_points, dim):
    test_points = generate_test_points(n_points, dim)
    passed_tests = 0
    
    for i, point in enumerate(test_points):
        point = jnp.array(point)  # 转换为 JAX 数组
        H = hessian_func(point)
        is_psd = is_positive_semidefinite(H)
        
        if is_psd:
            passed_tests += 1
        else:
            print(f"\n不通过测试点 {i+1}:")
            print(f"坐标: {point}")
            print(f"目标函数值: {objective(point):.6f}")
            print("Hessian 矩阵:")
            print(H)
            print("特征值:")
            print(jnp.linalg.eigvals(H))
    
    pass_rate = (passed_tests / n_points) * 100
    print(f"\n总体测试结果:")
    print(f"通过测试点数: {passed_tests}")
    print(f"总测试点数: {n_points}")
    print(f"通过率: {pass_rate:.2f}%")

# 运行测试
# np.random.seed(42)  # 设置随机种子以确保结果可重现
test_hessian_semidefinite(n_points=10000, dim=3)


不通过测试点 10:
坐标: [3.07208178 2.55022465 0.98631217]
目标函数值: 2126.055488
Hessian 矩阵:
[[2491797.20332209  412993.84222755  276372.45009708]
 [ 412993.84222755  137486.46658541   74398.23248801]
 [ 276372.45009708   74398.23248801   42217.50818969]]
特征值:
[ 2.59396432e+06+0.j  7.77733120e+04+0.j -2.36453010e+02+0.j]

不通过测试点 39:
坐标: [0.31202158 2.99778972 3.04990445]
目标函数值: 10960.162315
Hessian 矩阵:
[[  1417522.0506704    7264631.15386089  -2077925.49626359]
 [  7264631.1538609   37569598.26738119 -16447075.1070624 ]
 [ -2077925.49626356 -16447075.10706246  13689519.66982833]]
特征值:
[47157382.97525175+0.j  -158847.37682418+0.j  5678104.38945232+0.j]

不通过测试点 54:
坐标: [0.82914821 2.99785226 2.83625566]
目标函数值: 6817.560558
Hessian 矩阵:
[[ 167482.09520887  110718.85221388  672111.34674091]
 [ 110718.85221388 1388484.04080921 -164661.82836745]
 [ 672111.34674091 -164661.82836744 2898078.48386037]]
特征值:
[  -4183.75570151+0.j 3065502.5972435 +0.j 1392725.77833646+0.j]

不通过测试点 110:
坐标: [2.32907753 3.07110

In [77]:
import jax
import jax.numpy as jnp
from jax import grad, hessian
import numpy as np

# 启用 64 位精度
jax.config.update("jax_enable_x64", True)

d = 1

# 定义 A(x) 矩阵
def A(x):
    r = len(x)
    return jnp.array([[jnp.sin(i * x[j]) for i in range(1, r+1)] for j in range(r)])

# 定义目标函数
def objective(x):
    A_matrix = A(x)
    Omegas = np.arange(1, len(x) + 1)
    p = jnp.array(Omegas ** d * (-1 if d % 4 == 3 else 1))
    # p = jnp.array(np.random.uniform(0, 1, len(x)))
    try:
        b = jnp.linalg.solve(A_matrix.T, p)
        # 使用 l1 范数
        # return jnp.sum(jnp.abs(b))
        # 使用 l1 范数的平方
        # return jnp.sum(jnp.abs(b))**2
        # 使用 l2 范数
        # return jnp.sqrt(jnp.sum(jnp.abs(b)**2))
        # 使用 l2 范数的平方
        return jnp.sum(jnp.abs(b)**2)
    except:
        return jnp.inf

def Hessian_analysis(x, v):
    r = len(x)
    A_matrix = A(x)
    A_1 = np.array([[i * np.cos(i * x[j]) for i in range(1, r+1)] for j in range(r)])
    A_2 = np.array([[- (i**2) * np.sin(i * x[j]) for i in range(1, r+1)] for j in range(r)])
    Omegas = np.arange(1, len(x) + 1)
    p = jnp.array(Omegas ** d * (-1 if d % 4 == 3 else 1))
    b = jnp.linalg.solve(A_matrix.T, p)
    V = jnp.diag(v)
    C = A_1 @ jnp.linalg.inv(A_matrix)
    bbT = np.outer(b, b)
    E = C @ V @ C @ bbT
    E = E + C @ C.T @ V @ bbT
    E = E + C @ bbT @ V @ C
    E = E - V @ A_2 @ jnp.linalg.inv(A_matrix) @ bbT
    return 2*np.diag(E)

hessian_func = jax.jit(jax.hessian(objective))

def Hessian_jax(x_mu, v):
    H = hessian_func(jnp.array(x_mu))
    #  numpy 默认 v 都是行向量
    return H @ v.T

def test_Hessian_analysis(n_points, dim):
    test_points = generate_test_points(n_points, dim)
    passed_tests = 0
    
    for i, point in enumerate(test_points):
        v = np.random.random(len(point))
        value1 = Hessian_analysis(point, v)
        value2 = Hessian_jax(point, v)
        
        print(f"\nTest {i+1}:")
        print(f"Point: {point}")
        print(f"v: {v}")
        print(f"Hessian_analysis result: {value1}")
        print(f"Hessian_jax result: {value2}")
        
        if np.allclose(value1, value2, rtol=1e-5, atol=1e-8):
            print("Test PASSED")
            passed_tests += 1
        else:
            print("Test FAILED")
    
    print(f"\nSummary: Passed {passed_tests} out of {n_points} tests")

# Run the test
# np.random.seed(42)  # Set random seed for reproducibility
test_Hessian_analysis(n_points=10, dim=3)


Test 1:
Point: [2.21152961 1.53808429 0.76961175]
v: [0.52426541 0.23612086 0.24740012]
Hessian_analysis result: [ 0.04746673 -0.02722855 15.23608449]
Hessian_jax result: [ 0.04746673 -0.02722855 15.23608449]
Test PASSED

Test 2:
Point: [1.80701126 0.37913205 3.12438676]
v: [0.08030198 0.9938612  0.494642  ]
Hessian_analysis result: [  3352.60316171   8781.61687597 134409.76641456]
Hessian_jax result: [  3352.60316171   8781.61687597 134409.76641456]
Test PASSED

Test 3:
Point: [2.51035411 0.48074169 2.27792871]
v: [0.66678494 0.79414007 0.44563879]
Hessian_analysis result: [-90.31388262 231.78719515 191.74610144]
Hessian_jax result: [-90.31388262 231.78719515 191.74610144]
Test PASSED

Test 4:
Point: [1.77104105 2.29484341 1.05958115]
v: [0.98886087 0.53602773 0.38855937]
Hessian_analysis result: [ 215.36564192 -110.03706314  158.06545413]
Hessian_jax result: [ 215.36564192 -110.03706314  158.06545413]
Test PASSED

Test 5:
Point: [0.82630834 1.29993733 2.52636112]
v: [0.15675345 0.37

In [81]:
import numpy as np

def compute_eigenvalues(Hessian_analysis, x, n):
    # 构建 Hessian 矩阵
    H = np.zeros((n, n))
    for i in range(n):
        ei = np.zeros(n)
        ei[i] = 1
        H_ei = Hessian_analysis(x, ei)
        for j in range(n):
            ej = np.zeros(n)
            ej[j] = 1
            H[i, j] = np.dot(ej, H_ei)
    
    # 计算特征值
    eigenvalues = np.linalg.eigvals(H)
    
    return eigenvalues

# 假设我们有一个 n 维的问题
n = 3  # 这里以 3 维为例
x = np.array([3.07208178,2.55022465,0.98631217])  # 这是我们要计算 Hessian 的点

# 计算特征值
eigenvalues = compute_eigenvalues(Hessian_analysis, x, n)

print("Hessian 矩阵的特征值:", eigenvalues)

Hessian 矩阵的特征值: [ 2.59396376e+06  7.77733039e+04 -2.36452973e+02]


In [70]:
import jax
import jax.numpy as jnp
from jax import grad, hessian
import numpy as np

# 启用 64 位精度
jax.config.update("jax_enable_x64", True)

d = 1

# 定义 A(x) 矩阵
def A(x):
    r = len(x)
    return jnp.array([[jnp.sin(i * x[j]) for i in range(1, r+1)] for j in range(r)])

# 定义目标函数
def objective(x):
    A_matrix = A(x)
    Omegas = np.arange(1, len(x) + 1)
    p = jnp.array(Omegas ** d * (-1 if d % 4 == 3 else 1))
    # p = jnp.array(np.random.uniform(0, 1, len(x)))
    try:
        b = jnp.linalg.solve(A_matrix.T, p)
        # 使用 l1 范数
        # return jnp.sum(jnp.abs(b))
        # 使用 l1 范数的平方
        # return jnp.sum(jnp.abs(b))**2
        # 使用 l2 范数
        # return jnp.sqrt(jnp.sum(jnp.abs(b)**2))
        # 使用 l2 范数的平方
        return jnp.sum(jnp.abs(b)**2)
    except:
        return jnp.inf

def Hessian_analysis(x, v):
    r = len(x)
    A_matrix = A(x)
    A_1 = np.array([[i * np.cos(i * x[j]) for i in range(1, r+1)] for j in range(r)])
    A_2 = np.array([[- (i**2) * np.sin(i * x[j]) for i in range(1, r+1)] for j in range(r)])
    Omegas = np.arange(1, len(x) + 1)
    p = jnp.array(Omegas ** d * (-1 if d % 4 == 3 else 1))
    b = jnp.linalg.solve(A_matrix.T, p)
    V = jnp.diag(v)
    C = A_1 @ jnp.linalg.inv(A_matrix)
    bbT = np.outer(b, b)
    def frobenius_inner_product(A, B):
    # """计算两个矩阵的 Frobenius 内积"""
        return np.sum(A * B)
    term_1 = frobenius_inner_product(V, C @ V @ C @ bbT)
    term_2 = frobenius_inner_product(V, C @ C.T @ V @ bbT)
    term_3 = frobenius_inner_product(V, C @ bbT @ V @ C)
    term_4 = frobenius_inner_product(V, - V @ A_2 @ jnp.linalg.inv(A_matrix) @ bbT)
    print(f"term_1: {term_1}")
    print(f"term_2: {term_2}")
    print(f"term_3: {term_3}")
    print(f"term_4: {term_4}")
    return 2*np.sum(term_1 + term_2 + term_3 + term_4)

hessian_func = jax.jit(jax.hessian(objective))

def Hessian_jax(x_mu, v):
    H = hessian_func(jnp.array(x_mu))
    #  numpy 默认 v 都是行向量
    return np.dot(v, H @ v.T)

def test_Hessian_analysis(n_points, dim):
    test_points = generate_test_points(n_points, dim)
    passed_tests = 0
    
    for i, point in enumerate(test_points):
        v = np.random.random(len(point))
        value1 = Hessian_analysis(point, v)
        value2 = Hessian_jax(point, v)
        
        print(f"\nTest {i+1}:")
        print(f"Point: {point}")
        print(f"v: {v}")
        print(f"Hessian_analysis result: {value1}")
        print(f"Hessian_jax result: {value2}")
        
        if np.allclose(value1, value2, rtol=1e-5, atol=1e-8):
            print("Test PASSED")
            passed_tests += 1
        else:
            print("Test FAILED")
    
    print(f"\nSummary: Passed {passed_tests} out of {n_points} tests")

# Run the test
# np.random.seed(42)  # Set random seed for reproducibility
test_Hessian_analysis(n_points=1000, dim=3)

term_1: -2095.75126688425
term_2: 1731.7415764573288
term_3: -2095.7512668842483
term_4: 7381.853679187692

Test 1:
Point: [2.73458414 0.56880738 0.62473813]
v: [0.90404031 0.96301566 0.83886979]
Hessian_analysis result: 9844.185443753046
Hessian_jax result: 9844.185443752911
Test PASSED
term_1: 412983168333.2579
term_2: 485927032258.3828
term_3: 412983168333.25775
term_4: -184856375069.12085

Test 2:
Point: [3.12651126 1.51544969 3.06034362]
v: [0.12180087 0.53871398 0.38947622]
Hessian_analysis result: 2254073987711.555
Hessian_jax result: 2254073987711.3315
Test PASSED
term_1: -2.3348684980644587
term_2: 1.7925941927975764
term_3: -2.3348684980644587
term_4: 10.1977202679829

Test 3:
Point: [1.5467822  0.59162544 2.04030495]
v: [0.63844941 0.42237069 0.74795416]
Hessian_analysis result: 14.641154929303118
Hessian_jax result: 14.641154929303111
Test PASSED
term_1: 13690.314518180772
term_2: 13697.249838097581
term_3: 13690.314518180772
term_4: 603.236068275719

Test 4:
Point: [1.6062